In [1]:
import torch
from torch.utils.data import Dataset
import h5py
import json
import os
import pandas as pd
import random

from dataloader import PreLoadedReps_v2,PreLoadedReps

from models import Encoder, DecoderWithAttention,HIPT_LGP_FC

In [10]:
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
df_path='/home/ss4yd/vision_transformer/captioning_vision_transformer/prepared_prelim_data_tokenized_cls256.pickle'

In [4]:
pd.read_pickle(df_path).head()

,pid,tissue_type,notes,svs_path,patch_path,reps_path,tokens,p_tokens,caplens,dtype,idx_tokens
0,GTEX-1117F-0726,Heart - Atrial Appendage,"2 pieces, no abnormalities",/project/GutIntelligenceLab/ss4yd/gtex_data/ac...,/project/GutIntelligenceLab/ss4yd/gtex_data/pa...,/project/GutIntelligenceLab/ss4yd/gtex_data/hi...,"[2, pieces, ,, no, abnormalities]","[<start>, 2, pieces, ,, no, abnormalities, <en...",7,train,"[153, 1, 2, 3, 4, 5, 154, 0, 0, 0, 0, 0, 0, 0,..."
1,GTEX-1117F-1026,Lung,"2 pieces, moderate congestion/moderate to mark...",/project/GutIntelligenceLab/ss4yd/gtex_data/ac...,/project/GutIntelligenceLab/ss4yd/gtex_data/pa...,/project/GutIntelligenceLab/ss4yd/gtex_data/hi...,"[2, pieces, ,, moderate, congestion/moderate, ...","[<start>, 2, pieces, ,, moderate, congestion/m...",10,train,"[153, 1, 2, 3, 6, 152, 7, 8, 9, 154, 0, 0, 0, ..."
2,GTEX-1117F-1126,Liver,"2 pieces, subtotal massive hepatic necrosis",/project/GutIntelligenceLab/ss4yd/gtex_data/ac...,/project/GutIntelligenceLab/ss4yd/gtex_data/pa...,/project/GutIntelligenceLab/ss4yd/gtex_data/hi...,"[2, pieces, ,, subtotal, massive, hepatic, nec...","[<start>, 2, pieces, ,, subtotal, massive, hep...",9,train,"[153, 1, 2, 3, 152, 152, 10, 11, 154, 0, 0, 0,..."
3,GTEX-1117F-1226,Spleen,"2 pieces, marked congestion/autolysis",/project/GutIntelligenceLab/ss4yd/gtex_data/ac...,/project/GutIntelligenceLab/ss4yd/gtex_data/pa...,/project/GutIntelligenceLab/ss4yd/gtex_data/hi...,"[2, pieces, ,, marked, congestion/autolysis]","[<start>, 2, pieces, ,, marked, congestion/aut...",7,train,"[153, 1, 2, 3, 8, 12, 154, 0, 0, 0, 0, 0, 0, 0..."
4,GTEX-1117F-1326,Adipose - Visceral (Omentum),"2 pieces, diffuse mesothelial hyperplasia; ~10...",/project/GutIntelligenceLab/ss4yd/gtex_data/ac...,/project/GutIntelligenceLab/ss4yd/gtex_data/pa...,/project/GutIntelligenceLab/ss4yd/gtex_data/hi...,"[2, pieces, ,, diffuse, mesothelial, hyperplas...","[<start>, 2, pieces, ,, diffuse, mesothelial, ...",16,train,"[153, 1, 2, 3, 13, 14, 15, 16, 17, 18, 152, 19..."


In [5]:
ds=PreLoadedReps(df_path,'val')

ds.__getitem__(0)[0].shape

torch.Size([256, 384])

In [6]:
train_loader = torch.utils.data.DataLoader(
            PreLoadedReps_v2(df_path,'train'),
            batch_size=1, shuffle=True, num_workers=1, pin_memory=True)

In [7]:
train_loader.dataset.__getitem__(0)[0].shape

torch.Size([21, 256, 384])

In [12]:
encoder =HIPT_LGP_FC(freeze_4k=True, pretrain_4k='vit4k_xs_dino', n_classes=2)
encoder.to(device)

# of Patches: 196
Loading Pretrained Local VIT model...
Done!
Freezing Pretrained Local VIT model
Done


HIPT_LGP_FC(
  (local_vit): VisionTransformer4K(
    (phi): Sequential(
      (0): Linear(in_features=384, out_features=192, bias=True)
      (1): GELU()
      (2): Dropout(p=0.0, inplace=False)
    )
    (pos_drop): Dropout(p=0.0, inplace=False)
    (blocks): ModuleList(
      (0): Block(
        (norm1): LayerNorm((192,), eps=1e-06, elementwise_affine=True)
        (attn): Attention(
          (qkv): Linear(in_features=192, out_features=576, bias=True)
          (attn_drop): Dropout(p=0.0, inplace=False)
          (proj): Linear(in_features=192, out_features=192, bias=True)
          (proj_drop): Dropout(p=0.0, inplace=False)
        )
        (drop_path): Identity()
        (norm2): LayerNorm((192,), eps=1e-06, elementwise_affine=True)
        (mlp): Mlp(
          (fc1): Linear(in_features=192, out_features=768, bias=True)
          (act): GELU()
          (fc2): Linear(in_features=768, out_features=192, bias=True)
          (drop): Dropout(p=0.0, inplace=False)
        )
      )
 

In [14]:
for i, (imgs, caps, caplens) in enumerate(train_loader):
#     data_time.update(time.time() - start)

    # Move to GPU, if available
    imgs = imgs.to(device).squeeze(0)
    caps = caps.to(device)
    caplens = caplens.to(device)
    
    print(imgs.shape)

    # Forward prop.
    imgs = encoder(imgs)
    break

torch.Size([48, 256, 384])


/usr/local/lib/python3.8/site-packages/torch/nn/functional.py:3631: UserWarning: Default upsampling behavior when mode=bicubic is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  warnings.warn(
/usr/local/lib/python3.8/site-packages/torch/nn/functional.py:3679: UserWarning: The default behavior for interpolate/upsample with float scale_factor changed in 1.6.0 to align with other frameworks/libraries, and now uses scale_factor directly, instead of relying on the computed output size. If you wish to restore the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn(


In [16]:
imgs[1]

tensor([[0.0275, 0.0205, 0.0213, 0.0257, 0.0224, 0.0176, 0.0203, 0.0164, 0.0213,
         0.0169, 0.0193, 0.0233, 0.0203, 0.0238, 0.0191, 0.0229, 0.0143, 0.0249,
         0.0220, 0.0260, 0.0246, 0.0222, 0.0251, 0.0234, 0.0204, 0.0211, 0.0186,
         0.0214, 0.0194, 0.0235, 0.0168, 0.0206, 0.0252, 0.0206, 0.0185, 0.0171,
         0.0207, 0.0171, 0.0248, 0.0187, 0.0269, 0.0183, 0.0171, 0.0172, 0.0191,
         0.0210, 0.0159, 0.0190]], device='cuda:0', grad_fn=<SoftmaxBackward0>)